This notebook is used to create a csv file that includes my point projections in a format similar to fplreview.com. This csv file can then be used for making advanced team optimization plans using Sertalp B. Cay's repo "fpl-optimization".

In [1]:
latest_gameweek = 7

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import json

# Save team data
Info on your own team; starting point for the optimization.

In [3]:
# get login credentials for fetching team data
file_path = Path('../../login_credentials/fpl_login.json')
#file_path = Path('../../login_credentials/fpl_login_2.json')
login_credentials = pd.read_json(file_path, typ='series')

In [4]:
# utility function for fetching team data
from src.utils import fetch_my_team

In [ ]:
# fetch my team data
my_team = fetch_my_team(login_credentials.user_name, login_credentials.password, login_credentials.team_id)
my_team

In [6]:
# save team data to fpl-optimization repo
file_path = Path('../../../repos/fpl-optimization/data/team.json')
with open(file_path, 'w') as f:
    json.dump(my_team, f)

# Edit projections into fplreview format 
### (to be used with fpl-optimization repo solver)

In [ ]:
filepath = Path(f'../data/predictions/gameweek{latest_gameweek}.csv')
projections = pd.read_csv(filepath, index_col=0)
if 'id' in projections.columns:
    projections = projections.drop('id', axis=1)
display(projections.head())
display(projections.shape)

In [ ]:
if latest_gameweek>0:
    filepath = Path('../data/fpl_df.csv')
elif latest_gameweek==0:
    filepath = Path('../data/fpl_df_preseason_online.csv')
else:
    print('Check latest_gameweek!')

fpl_df = pd.read_csv(filepath, index_col=0, low_memory=False)
fpl_df = fpl_df[fpl_df.season=='24-25']
display(fpl_df.head())
display(fpl_df.shape)

In [ ]:
df = fpl_df.groupby('name').last().reset_index()[['id', 'name','points_per_game', 'total_points',]]
#df['id'] = df.id.astype(int)
df['games_played'] = np.round(np.where(df['points_per_game']!=0, df['total_points'] / df['points_per_game'], 0),0)
display(df.head())
display(df.shape)

In [ ]:
# drop duplicate players (some players get new spelling for their name during the season causing duplicates)
duplicate_ids = df.loc[df.id.duplicated(), 'id'].unique()
for id in duplicate_ids:
    ix = df.loc[df.id==id, 'games_played'].idxmin()
    df = df.drop(ix)
display(df.head())
display(df.shape)

In [ ]:
# drop unneccesary columns
df = df.drop(['points_per_game', 'total_points','games_played'], axis=1)
display(df.head())
display(df.shape)

In [ ]:
# merge id info to projections
projections = projections.merge(df, on='name', how='left')
projections.head()

In [13]:
# add xMins variable that is needed later
projections['xMins'] = 90

In [ ]:
projections[['id', 'expected_points', 'xMins', 'gameweek']]

In [ ]:
projections_pivot = (
    projections
    .pivot_table(
    columns=['gameweek',],
    index='id', 
    values=['expected_points','xMins'], 
    aggfunc='sum'
    )
)

projections_pivot.head()

In [ ]:
new_cols = []
for col in projections_pivot.columns:
    if col[0] == 'expected_points':
        new_col = str(col[1]) + '_Pts'
        new_cols.append(new_col)
    elif col[0] == 'xMins':
        new_col = str(col[1]) + '_xMins'
        new_cols.append(new_col)

projections_pivot.columns = new_cols
projections_pivot = projections_pivot.reset_index()
projections_pivot = projections_pivot.rename(columns={'id':'ID'})

projections_pivot.head()

In [33]:
filepath = Path('../../../repos/fpl-optimization/data/fplreview.csv')
projections_pivot.to_csv(filepath)

### Option to make manual changes, e.g., to xPts (based on injuries)

### adjust points

If model is underestimating e.g. Haaland currently, test how many points more he would need to be getting in order to be drafed to the team in the optimization.

In [ ]:
projections.loc[projections['name'].str.contains('Have'), 'id'].unique()
#projections.loc[projections['name'].str.contains('Palme'), 'id'].unique()

In [ ]:
player_id = 4
display(projections.loc[projections.id==player_id, 'name'].unique())
display(projections_pivot[projections_pivot.ID==player_id])

In [ ]:
pts_cols = [col for col in projections_pivot.columns if 'Pts' in col]
projections_pivot.loc[projections_pivot.ID==player_id, pts_cols].sum(axis=1) / 10

In [ ]:
gameweeks = np.arange(latest_gameweek+1,latest_gameweek+11)
add_constant = 1.5

for i in range(len(gameweeks)):
    projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts'] = \
        projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts'] + add_constant

display(projections_pivot[projections_pivot.ID==player_id])

### Odegaard

In [ ]:
projections.loc[projections['name'].str.contains('degaard'), 'id'].unique()

In [ ]:
player_id = 13
display(projections.loc[projections.id==player_id, 'name'].unique())
display(projections_pivot[projections_pivot.ID==player_id])

In [ ]:
np.arange(latest_gameweek+1,latest_gameweek+3)

In [ ]:
gameweeks = np.arange(latest_gameweek+1,latest_gameweek+3)
weights = [0.5, 0.75]

for i in range(len(gameweeks)):
    projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts'] = \
        weights[i]*projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts']

display(projections_pivot[projections_pivot.ID==player_id])

### De Bruyne

In [ ]:
projections.loc[projections['name'].str.contains('Bruy'), 'id'].unique()

In [ ]:
player_id = 345
display(projections.loc[projections.id==player_id, 'name'].unique())
display(projections_pivot[projections_pivot.ID==player_id])

In [ ]:
gameweeks = [8,9]
weights = [0.5,0.75]

for i in range(len(gameweeks)):
    projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts'] = \
        weights[i]*projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts']

display(projections_pivot[projections_pivot.ID==player_id])

### Pedro

In [ ]:
projections.loc[projections['name'].str.contains('Pedro'), 'id'].unique()

In [ ]:
player_id = 129
display(projections.loc[projections.id==player_id, 'name'].unique())
display(projections_pivot[projections_pivot.ID==player_id])

In [ ]:
gameweeks = [8,9,10,11]
weights = [0,0,0,0]

for i in range(len(gameweeks)):
    projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts'] = \
        weights[i]*projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts']

display(projections_pivot[projections_pivot.ID==player_id])

### Turner

In [ ]:
projections.loc[projections['name'].str.contains('Turner'), 'id'].unique()

In [ ]:
player_id = 28
projections_pivot[projections_pivot.ID==player_id]

In [ ]:
gameweeks = np.arange(latest_gameweek+1,39)
weights = np.zeros(len(gameweeks))

for i in range(len(gameweeks)):
    projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts'] = \
        weights[i]*projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts']

display(projections_pivot[projections_pivot.ID==player_id])

### Ederson

In [ ]:
projections.loc[projections['name'].str.contains('Eders'), 'id'].unique()

In [ ]:
player_id = 352
projections_pivot[projections_pivot.ID==player_id]

In [ ]:
gameweeks = np.arange(latest_gameweek+1,39)
weights = np.zeros(len(gameweeks))

for i in range(len(gameweeks)):
    projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts'] = \
        weights[i]*projections_pivot.loc[projections_pivot.ID==player_id, f'{gameweeks[i]}_Pts']

display(projections_pivot[projections_pivot.ID==player_id])

In [ ]:
# save data
filepath = Path('../../../repos/fpl-optimization/data/fplreview.csv')
projections_pivot.to_csv(filepath)

# Other stuff

In [ ]:
# check ids
projections.loc[projections['name'].str.contains('Havertz'), ['id','name']]

In [ ]:
projections.loc[projections.id==491, ['name','team_name']]

In [ ]:
# banned players from solver 
projections.loc[projections.id.isin([69, 110, 13, 345, 333, 25, 129]), 'name'].unique()

In [ ]:
projections[projections.id==24]